## Analysis
- Seasons of interest - June, July, August, September.
- Districts of interest - Kolhapur, Latur

- A new LSTM model to predict rainfall.

### Import libraries

In [2]:
import warnings
warnings.filterwarnings('ignore')

import os
import shutil
import numpy as np
import pandas as pd
import seaborn as sns
import plotly.graph_objs as go
import plotly.offline as py
import matplotlib
import matplotlib.pyplot as plt
from mpl_toolkits.basemap import Basemap
import matplotlib.pyplot as plt
from matplotlib.patches import Polygon
import matplotlib.patches as mpatches 
from matplotlib.collections import PatchCollection
import plotly.figure_factory as ff
from IPython.display import HTML, display
from IPython.core import display as ICD
from plotly.offline import init_notebook_mode, iplot
init_notebook_mode(connected=True)

import Artificial_Neural_Networks as ANN
import ARIMA

import math
from itertools import groupby
%matplotlib inline
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.utils import plot_model
from keras.layers import Conv2D, MaxPooling2D, Flatten
from keras.layers import Input, LSTM, Embedding, Dense
from keras.models import Model, Sequential
from keras.layers.merge import concatenate
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot

from importlib import reload
import itertools

Using TensorFlow backend.


### Useful functions

In [3]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error

def mean_absolute_percentage_error(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

def root_mean_squared_error(y_true, y_pred):
    mse = mean_squared_error(y_true, y_pred)
    rmse = np.sqrt(mse)
    return rmse

def calculate_performance(y_true, y_pred):
    mse = mean_squared_error(y_true, y_pred)
    mae = mean_absolute_error(y_true, y_pred)
    mape = mean_absolute_percentage_error(y_true, y_pred)
    rmse = root_mean_squared_error(y_true, y_pred)
    return round(mse, 3), round(mae, 3), round(mape, 3), round(rmse, 3)

### Dataset

In [4]:
PATH = 'Dataset/rainfall_data_1901_to_2002.xlsx'
data = pd.read_excel(PATH)

### Preprocess data

In [5]:
data = data.drop(columns='vlookup')
data = data[data['Year'].notnull()]
data['Year'] = data.Year.astype('int')
data.index = range(len(data))

m_data = data[data['State'] == 'Maharashtra']
m_data = m_data.drop(columns='State')

districts = m_data.District.unique()
years = list(range(1901, 2003))
months = data.columns[3:]
year_month = [str(year) + '_' + month for year in years for month in months]
dates = pd.date_range(start='1901-01', freq='MS', periods=len(years)*12)

maharashtra_data = pd.DataFrame({'Year_Month': year_month})
maharashtra_data['Date'] = dates
maharashtra_data[['Year', 'Month']] = maharashtra_data['Year_Month'].str.split('_', n=1, expand=True)
maharashtra_data = maharashtra_data.drop(columns=['Year_Month'])

for district in districts:
    df = m_data[m_data.District == district].drop(columns=['District', 'Year'])
    df = df.as_matrix().reshape((len(years) * len(months), 1))[:,0]
    maharashtra_data[district] = df

maharashtra_data.head()

,Date,Year,Month,Ahmadnagar,Akola,Amravati,Aurangabad,Bhandara,Bid,Buldana,...,Nashik,Osmanabad,Parbhani,Pune,Sangli,Satara,Solapur,Wardha,Washim,Yavatmal
0,1901-01-01,1901,Jan,2.510,34.202,35.651,10.922,23.397,16.647,31.455,...,5.063,15.500,33.207,0.922,0.138,0.197,4.496,45.050,32.485,22.031
1,1901-02-01,1901,Feb,11.489,1.099,6.822,4.362,63.844,1.916,0.823,...,1.609,2.784,4.997,7.195,0.537,0.525,3.667,28.220,3.607,16.198
2,1901-03-01,1901,Mar,11.325,30.002,36.103,25.161,33.563,27.287,28.448,...,11.196,11.333,31.625,5.105,13.090,9.566,10.330,26.398,26.557,29.318
3,1901-04-01,1901,Apr,33.931,10.248,10.636,12.714,61.560,33.211,13.902,...,7.838,34.814,41.941,35.949,50.077,30.110,59.854,29.228,18.127,30.521
4,1901-05-01,1901,May,30.401,2.891,4.173,34.244,13.665,59.027,9.397,...,7.475,52.792,31.794,36.650,78.994,65.226,50.892,8.619,6.700,13.193


In [6]:
m_data = maharashtra_data.copy()

In [7]:
m_data.head()

,Date,Year,Month,Ahmadnagar,Akola,Amravati,Aurangabad,Bhandara,Bid,Buldana,...,Nashik,Osmanabad,Parbhani,Pune,Sangli,Satara,Solapur,Wardha,Washim,Yavatmal
0,1901-01-01,1901,Jan,2.510,34.202,35.651,10.922,23.397,16.647,31.455,...,5.063,15.500,33.207,0.922,0.138,0.197,4.496,45.050,32.485,22.031
1,1901-02-01,1901,Feb,11.489,1.099,6.822,4.362,63.844,1.916,0.823,...,1.609,2.784,4.997,7.195,0.537,0.525,3.667,28.220,3.607,16.198
2,1901-03-01,1901,Mar,11.325,30.002,36.103,25.161,33.563,27.287,28.448,...,11.196,11.333,31.625,5.105,13.090,9.566,10.330,26.398,26.557,29.318
3,1901-04-01,1901,Apr,33.931,10.248,10.636,12.714,61.560,33.211,13.902,...,7.838,34.814,41.941,35.949,50.077,30.110,59.854,29.228,18.127,30.521
4,1901-05-01,1901,May,30.401,2.891,4.173,34.244,13.665,59.027,9.397,...,7.475,52.792,31.794,36.650,78.994,65.226,50.892,8.619,6.700,13.193


### Preprocess data

In [8]:
def LSTM_model(num_of_previous_months, hidden_nodes_months, 
               num_of_previous_years, hidden_nodes_years, output_nodes):
    
    visible1 = Input((num_of_previous_months, 1))
#     visible1 = Input((1, num_of_previous_months))
    extract1 = LSTM(hidden_nodes_months, activation='relu')(visible1)

    visible2 = Input((num_of_previous_years, 1))
#     visible2 = Input((1, num_of_previous_years))
    extract2 = LSTM(hidden_nodes_years, activation='relu')(visible2)

    merge = concatenate([extract1, extract2])
    output = Dense(output_nodes)(merge)
    
    model = Model(inputs = [visible1, visible2], outputs = output)
    model.compile(loss='mean_squared_error', optimizer='adam')
    
    plot_model(model, 'Functional_LSTM.png', show_shapes=True, show_layer_names=True)
    
    return model

In [9]:
def preprocess_data(m_data, district, month, num_of_prev_months, num_of_prev_years):
    
#     rainfall_season_data = m_data[['Date', 'Year', 'Month'] + districts_of_interest]
    rainfall_data = m_data[['Date', 'Year', 'Month', district]]
    month_data = rainfall_data[rainfall_data.Month == month]
    
    start_year = int(rainfall_data.Year.min())
    last_year = int(rainfall_data.Year.max())
    current_year = start_year + num_of_previous_years
    month_data_index = month_data.index
    
    train_data_input_1 = []
    for index in month_data_index[num_of_previous_years:]:
        data = list(rainfall_data.iloc[index - num_of_previous_months:index][district])
        train_data_input_1.append(data)
    train_data_input_1 = np.array(train_data_input_1)
    shape = train_data_input_1.shape
    train_data_input_1 = train_data_input_1.reshape(shape[0], shape[1], 1)
#     train_data_input_1 = train_data_input_1.reshape(shape[0], 1, shape[1])
    
    month_data_prep = list(month_data[district])
    train_data_input_2 = []
    for i in range(0, len(month_data_prep) - num_of_previous_years):
        data = month_data_prep[i:i+num_of_previous_years]
        train_data_input_2.append(data)
    train_data_input_2 = np.array(train_data_input_2)
    shape = train_data_input_2.shape
    train_data_input_2 = train_data_input_2.reshape(shape[0], shape[1], 1)
#     train_data_input_2 = train_data_input_2.reshape(shape[0], 1, shape[1])
    
    y_train = list(month_data.iloc[num_of_previous_years:][district])
    y_train = np.array(y_train)
    y_train = np.reshape(y_train, (y_train.shape[0], 1))
    
    return train_data_input_1, train_data_input_2, y_train

In [10]:
def split_and_train_LSTM(model, input_1, input_2, y_train_main, future_steps, epochs, batch_size):
    X_train_input_1, X_test_input_1, y_train, y_test = train_test_split(input_1, y_train_main, 
                                                                    test_size=future_steps, random_state=42)
    X_train_input_2, X_test_input_2, y_train, y_test = train_test_split(input_2, y_train_main, 
                                                                    test_size=future_steps, random_state=42)
    
    model.fit([X_train_input_1, X_train_input_2], y_train, epochs=epochs, batch_size=batch_size, verbose=1, shuffle=True)
    return model, X_test_input_1, X_test_input_2, y_test

In [11]:
def predict_LSTM(model, X_test_input_1, X_test_input_2):
    y_pred = model.predict([X_test_input_1, X_test_input_2])
    return y_pred

In [12]:
def Long_Short_Term_Memory(data, district, month, num_of_prev_months, num_of_prev_years, hidden_nodes_months, hidden_nodes_years, epochs, batch_size, future_steps):
    
    model = LSTM_model(num_of_prev_months, hidden_nodes_months, num_of_prev_years, hidden_nodes_years, output_nodes)
    train_data_input_1, train_data_input_2, y_train_main = preprocess_data(m_data, district, month, num_of_prev_months, num_of_prev_years)
    model, X_test_input_1, X_test_input_2, y_test = split_and_train_LSTM(model, train_data_input_1, train_data_input_2, y_train_main, future_steps, epochs, batch_size)
    y_pred = predict_LSTM(model, X_test_input_1, X_test_input_2)
    return model, y_pred

In [13]:
future_steps = 5

# number_of_previous_months, number_of_previous_years, hidden_nodes_months, hidden_nodes_years, epochs, batch_size, future_steps
parameters_LSTM = [[3,6,9,12], [4,6,8], [6,8,10,12], [5,7,8], [100], [10], [future_steps]]
num_of_previous_months = 12
num_of_previous_years = 7
hidden_nodes_months = 12
hidden_nodes_years = 5
output_nodes = 1

In [16]:
Long_Short_Term_Memory(m_data, district, month, num_of_previous_months, num_of_previous_years,
                      hidden_nodes_months, hidden_nodes_years, 250, 10, future_steps)

Epoch 1/400
90/90 [==============================] - 1s 13ms/step - loss: 394880.4097
Epoch 2/400
90/90 [==============================] - 0s 1ms/step - loss: 393548.3889
Epoch 3/400
90/90 [==============================] - 0s 1ms/step - loss: 390958.3785
Epoch 4/400
90/90 [==============================] - 0s 1ms/step - loss: 382849.9931
Epoch 5/400
90/90 [==============================] - 0s 1ms/step - loss: 375145.2188
Epoch 6/400
90/90 [==============================] - 0s 1ms/step - loss: 367774.1632
Epoch 7/400
90/90 [==============================] - 0s 1ms/step - loss: 361752.2691
Epoch 8/400
90/90 [==============================] - 0s 1ms/step - loss: 355025.8976
Epoch 9/400
90/90 [==============================] - 0s 1ms/step - loss: 348287.3003
Epoch 10/400
90/90 [==============================] - 0s 1ms/step - loss: 339520.1424
Epoch 11/400
90/90 [==============================] - 0s 1ms/step - loss: 328427.0729
Epoch 12/400
90/90 [==============================] - 0s 1ms/s

90/90 [==============================] - 0s 2ms/step - loss: 63908.9319
Epoch 98/400
90/90 [==============================] - 0s 2ms/step - loss: 62561.5469
Epoch 99/400
90/90 [==============================] - 0s 2ms/step - loss: 64069.7001
Epoch 100/400
90/90 [==============================] - 0s 2ms/step - loss: 61869.5920
Epoch 101/400
90/90 [==============================] - 0s 1ms/step - loss: 62023.8576
Epoch 102/400
90/90 [==============================] - 0s 934us/step - loss: 64158.2969
Epoch 103/400
90/90 [==============================] - 0s 983us/step - loss: 59440.3546
Epoch 104/400
90/90 [==============================] - 0s 1ms/step - loss: 56516.1311
Epoch 105/400
90/90 [==============================] - 0s 949us/step - loss: 54002.0467
Epoch 106/400
90/90 [==============================] - 0s 913us/step - loss: 52802.2120
Epoch 107/400
90/90 [==============================] - 0s 1ms/step - loss: 51444.0518
Epoch 108/400
90/90 [==============================] - 0s 1ms/

90/90 [==============================] - 0s 1ms/step - loss: 47902.7865
Epoch 193/400
90/90 [==============================] - 0s 1ms/step - loss: 48558.0306
Epoch 194/400
90/90 [==============================] - 0s 1ms/step - loss: 71968.3342
Epoch 195/400
90/90 [==============================] - 0s 2ms/step - loss: 98178.4436
Epoch 196/400
90/90 [==============================] - 0s 2ms/step - loss: 91208.0701
Epoch 197/400
90/90 [==============================] - 0s 2ms/step - loss: 86704.1155
Epoch 198/400
90/90 [==============================] - 0s 1ms/step - loss: 104580.7565
Epoch 199/400
90/90 [==============================] - 0s 1ms/step - loss: 79038.8368
Epoch 200/400
90/90 [==============================] - 0s 1ms/step - loss: 78783.7374
Epoch 201/400
90/90 [==============================] - 0s 1ms/step - loss: 93620.5126
Epoch 202/400
90/90 [==============================] - 0s 1ms/step - loss: 89701.1984
Epoch 203/400
90/90 [==============================] - 0s 1ms/step 

90/90 [==============================] - 0s 1ms/step - loss: 62454.0790
Epoch 288/400
90/90 [==============================] - 0s 898us/step - loss: 61760.3360
Epoch 289/400
90/90 [==============================] - 0s 1ms/step - loss: 59499.5017
Epoch 290/400
90/90 [==============================] - 0s 982us/step - loss: 61034.3952
Epoch 291/400
90/90 [==============================] - 0s 1ms/step - loss: 60332.3058
Epoch 292/400
90/90 [==============================] - 0s 1ms/step - loss: 60431.6233
Epoch 293/400
90/90 [==============================] - 0s 940us/step - loss: 59646.1443
Epoch 294/400
90/90 [==============================] - 0s 1ms/step - loss: 58334.7255
Epoch 295/400
90/90 [==============================] - 0s 1ms/step - loss: 57474.5768
Epoch 296/400
90/90 [==============================] - 0s 1ms/step - loss: 56195.6842
Epoch 297/400
90/90 [==============================] - 0s 1ms/step - loss: 55979.2374
Epoch 298/400
90/90 [==============================] - 0s 982u

90/90 [==============================] - 0s 1ms/step - loss: 74986.8641
Epoch 383/400
90/90 [==============================] - 0s 991us/step - loss: 74740.0356
Epoch 384/400
90/90 [==============================] - 0s 951us/step - loss: 72457.7118
Epoch 385/400
90/90 [==============================] - 0s 940us/step - loss: 70973.5673
Epoch 386/400
90/90 [==============================] - 0s 1ms/step - loss: 70271.7101
Epoch 387/400
90/90 [==============================] - 0s 1ms/step - loss: 69660.2381
Epoch 388/400
90/90 [==============================] - 0s 1ms/step - loss: 68509.1641
Epoch 389/400
90/90 [==============================] - 0s 984us/step - loss: 68255.2867
Epoch 390/400
90/90 [==============================] - 0s 977us/step - loss: 67791.4360
Epoch 391/400
90/90 [==============================] - 0s 1ms/step - loss: 66962.2884
Epoch 392/400
90/90 [==============================] - 0s 958us/step - loss: 66210.7105
Epoch 393/400
90/90 [==============================] - 0

(<keras.engine.training.Model at 0x7ff1dd222780>, array([[494.6607],
        [527.5482],
        [718.7484],
        [402.4953],
        [613.7914]], dtype=float32))

In [ ]:
districts_of_interest = ['Kolhapur', 'Latur']
months_of_interest = ['Jun', 'Jul', 'Aug', 'Sep']

In [15]:
district = 'Kolhapur'
month = 'Jun'

In [ ]:
model = LSTM_model(num_of_previous_months, hidden_nodes_months, num_of_previous_years, hidden_nodes_years, output_nodes)
train_data_input_1, train_data_input_2, y_train_main = preprocess_data(m_data, district, month, 12, 7)

In [ ]:
train_data_input_1.shape, train_data_input_2.shape, y_train_main.shape

In [ ]:
train_data_input_1.shape

In [ ]:
model, X_test_input_1, X_test_input_2, y_test = split_and_train_LSTM(model, train_data_input_1, train_data_input_2, future_steps, 100, 5)

In [ ]:
y_pred = predict_LSTM(model, X_test_input_1, X_test_input_2)

In [ ]:
y_pred

In [ ]:
y_test

In [ ]:
month = 'Jun'
preprocess_data(rainfall_data, month, num_of_previous_months, num_of_previous_years)

In [ ]:
# for district in districts_of_interest:
#     temp_data = rainfall_season_data[['Date', 'Year', 'Month', district]]
#     for month in months_of_interest:
#         df = temp_data[temp_data.Month == month]

In [ ]:
# LSTM_model(num_of_previous_months, hidden_nodes_months, num_of_previous_years, hidden_nodes_years, output_nodes)